In [2]:
import torch
import numpy as np
from stable_baselines3 import SAC


import os, shutil, sys
sys.path.append(os.path.abspath('./env'))
sys.path.append(os.path.abspath('./common'))

from env.env_move_sector  import HumanMoveSectorAction
import common.mlflow_sb3_helper as mlf
import common.tensorboard_sb3_helper as tnb
USE_MLFLOW = False

from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
exp_params = {
    'env_name': '',
    'algorithm_name': '',
    'exp_id': 0,
    'exp_name': f'exp_{datetime.now(TZ).strftime("%d%m_%H%M%S")}',
    'seed': 21,
    'net': {
        'activation': 'ReLU',
        'pi': [512, 256, 128],
        'qf': [512, 256, 128],
    },
    'training': {
        'iteration_count': 1,
        'episode_count': 39000,
        'policy': 'MlpPolicy',
        'learning_rate': 0.001,
        'buffer_size': 1500000,
        'learning_starts': 100,
        'batch_size': 256,
        'tau': 0.001,
        'gamma': 0.95,
        'verbose': 0,
        'device': device,
    },
    'options':{
        'finish_dist':100,
        'start_dist':110,
        'delta_phi':0
    },
    'validation': {
        'validate_agent_every_n_eps': 20000,
        'log_interval': 10
    },
}


In [4]:
log_helper = {}
log_path = {}

if USE_MLFLOW:
    log_helper = mlf.MLflowServerHelper("http://192.168.0.206:2670", False)
else:
    log_helper = tnb.TensorboardHelper("",False)

In [5]:
experiment_id = 390
log_path = {
 'SAC': f'mlflow-artifacts:/{experiment_id}/66c5989f115c4c71a1588d309e6a1db9/artifacts/env_MoveSector_RanTP_IgnoreObst_exp_1510_180214/sb3/model.zip'
 }

In [ ]:

load_path = os.path.join(str(experiment_id))
if  os.path.isdir(load_path) == False:
    os.mkdir(load_path)
load_path

In [ ]:
loadel_models = {}

for model_name, path in log_path.items():
    loadel_models[model_name] =  f'{load_path}/{model_name}/'
    log_helper.load_artifact(path,loadel_models[model_name])

In [8]:
env = HumanMoveSectorAction(target_point_rand=True, object_ignore=False)
env_render = HumanMoveSectorAction(target_point_rand=True, object_ignore=False, render_mode='rgb_array')

In [ ]:
exp_params['env_name'] = env.name()
exp_name = 'env_' + exp_params['env_name'] + '_' + exp_params['exp_name']
experiment_id = log_helper.new_experiment(exp_name)
exp_params['exp_id'] = experiment_id
print(exp_name)

In [ ]:
#!!! Дорасчет в СТАРОМ эксперименте
exp_name = log_helper.get_experiment(experiment_id)
print(exp_name)
exp_params['exp_id'] = experiment_id
exp_params['env_name'] = env.name()

In [ ]:

exp_params['algorithm_name'] = 'SAC'

path = loadel_models['SAC']
print(path)
name = 'SAC'

is_first = True

for i in range(exp_params['training']['iteration_count']):


    exp_params['seed'] = int(datetime.now(TZ).strftime("%H%M%S"))

    if is_first:
        model = SAC.load(f'{path}/model.zip', env=env, device=device)
        is_first = False
    else:
        model = SAC.load(f'{exp_name}/model.zip', env=env, device=device)


    art_loc, exp_name, run_id = log_helper.learn_and_fix(
                                model=model, 
                                env= env_render,
                                run_name=f'{name}_{i+1}',
                                episode_count = exp_params['training']['episode_count'],
                                parameters=exp_params,
                                experiment_id=experiment_id,
                                checkpoint_interval=exp_params['validation']['validate_agent_every_n_eps'],
                                log_interval=exp_params['validation']['log_interval'],
                                video_episods=3,
                                video_fps=120                                   
                                )

    log_path[name] = f'{art_loc}/{run_id}/artifacts/{exp_name}/sb3/model.zip'



In [ ]:
log_path

In [ ]:
shutil.rmtree(os.path.join(load_path))